# **Instructions**:

## Step 1: Setup
- Make sure you have Python installed on your system. If not, download and install it from the official Python website.
- Install the necessary Python libraries by running `pip install numpy win32gui win32ui win32con pillow`.

## Step 2: Open your game
- Open the game that you want to perform detections on.

## Step 3: Identify the game window
- Look at the title bar of the game window. This is usually at the top of the window and contains the name of the game.

## Step 4: Update the script
- Open the `1_generate_dataset.ipynb` file in your Python IDE (like PyCharm).
- Find the line that says `window_name = "Old School RuneScape"`.
- Replace `"Old School RuneScape"` with the name of your game's title bar. Make sure to keep the quotes around the name.

## Step 5: Run the script
- Run all cells in the `1_generate_dataset.ipynb` file. This will start generating your dataset in the `images` folder.

## Step 6: Repeat for other scripts
- Repeat steps 4 and 5 for the other scripts (`2_label_dataset.ipynb`, `3_train_model.ipynb`, `4_yolo_opencv_detector.ipynb`), making sure to update the `window_name` variable in each script.

## Step 7: Check the output
- After running all scripts, check the `images` folder to see the generated dataset.
- If you encounter any errors, make sure you followed all steps correctly. If the error persists, seek help from a knowledgeable friend or online community.

In [2]:
import numpy as np
import win32gui, win32ui, win32con
from PIL import Image
from time import sleep
import os

class WindowCapture:
    """
    A class used to capture screenshots from a specified window.

    ...

    Attributes
    ----------
    w : int
        width of the window
    h : int
        height of the window
    hwnd : int
        handle to the window
    cropped_x : int
        cropped width due to border
    cropped_y : int
        cropped height due to title bar

    Methods
    -------
    get_screenshot():
        Captures a screenshot from the specified window.
    generate_image_dataset():
        Generates a dataset of images by continuously capturing screenshots from the window.
    get_window_size():
        Returns the size of the window.
    save_image(img, path):
        Saves the image to the specified path.
    """

    def __init__(self, window_name):
        """Initializes WindowCapture with a window name."""
        self.hwnd = win32gui.FindWindow(None, window_name)
        if not self.hwnd:
            raise Exception('Window not found: {}'.format(window_name))

        window_rect = win32gui.GetWindowRect(self.hwnd)
        self.w = window_rect[2] - window_rect[0]
        self.h = window_rect[3] - window_rect[1]

        border_pixels = 8
        titlebar_pixels = 30
        self.w = self.w - (border_pixels * 2)
        self.h = self.h - titlebar_pixels - border_pixels
        self.cropped_x = border_pixels
        self.cropped_y = titlebar_pixels

    def get_screenshot(self):
        """Captures a screenshot from the specified window."""
        wDC = win32gui.GetWindowDC(self.hwnd)
        dcObj = win32ui.CreateDCFromHandle(wDC)
        cDC = dcObj.CreateCompatibleDC()
        dataBitMap = win32ui.CreateBitmap()
        dataBitMap.CreateCompatibleBitmap(dcObj, self.w, self.h)
        cDC.SelectObject(dataBitMap)
        cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)

        signedIntsArray = dataBitMap.GetBitmapBits(True)
        img = np.frombuffer(signedIntsArray, dtype='uint8')
        img.shape = (self.h, self.w, 4)

        dcObj.DeleteDC()
        cDC.DeleteDC()
        win32gui.ReleaseDC(self.hwnd, wDC)
        win32gui.DeleteObject(dataBitMap.GetHandle())

        img = img[...,:3]
        img = np.ascontiguousarray(img)

        return img

    def generate_image_dataset(self):
        """Generates a dataset of images by continuously capturing screenshots from the window."""
        if not os.path.exists("images"):
            try:
                os.mkdir("images")
            except OSError:
                print ("Creation of the directory 'images' failed")
                return
        while(True):
            img = self.get_screenshot()
            self.save_image(img, f"./images/img_{len(os.listdir('images'))}.jpg")
            sleep(0.3)

    def get_window_size(self):
        """Returns the size of the window."""
        return (self.w, self.h)

    def save_image(self, img, path):
        """Saves the image to the specified path."""
        im = Image.fromarray(img[..., [2, 1, 0]])
        im.save(path)

In [4]:
# Execute this cell to generate a dataset of images for the specified window.
window_name = "Old School RuneScape"  # Replace with the name of your game window

# Create an instance of the WindowCapture class
wincap = WindowCapture(window_name)

# Start generating the image dataset
wincap.generate_image_dataset()

C:\Users\pkmaz\AppData\Local\Temp\ipykernel_24960\3344924937.py:32: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  img = np.fromstring(signedIntsArray, dtype='uint8')


KeyboardInterrupt: 